In [2]:
import pyvisa
import numpy as np
import time
import pandas as pd
import os
import CVCommands
from simple_pid import PID
from datetime import datetime
from scipy.optimize import curve_fit, minimize, leastsq
from pyvisa import ResourceManager, constants
from matplotlib import pyplot as plt
from tqdm import tqdm, trange
from usb.core import find as finddev

In [5]:
sampName = 'CS519C1'

#Values for CIGS fine scan
V_i = 0.95 #Initial bias V (V)
V_f = 0.0 #Final bias V (V)
V_step = 30 #Number of steps

freq_i = 3 #Initial freq order of magnitude (Hz)
freq_f = 5 #Final freq order of magnitude (Hz)
freq_step = 5 #Number of steps

ampl_i = 0.05 #Initial V amplitude (V)
ampl_f = 0.3 #Final V amplitude (V)
ampl_step = 15 #Number of steps
num_avg = 5 #Number of measurements to average

delay = .1 #Delay between changing to new value\

holdbias = True #Toggle to hold device at bias before measurement
hold_val = V_i #Voltage to hold at in V
hold_time = 30 #Time to hold voltage in s



ϵ = 25 #relative permitivity
ϵo=8.85e-12 #permitivity of free space (m-3 kg-1 s4 A2)
q=1.602e-19 #elementary charge (A.s-1)
A = 0.17*1e-4 #cell area (m2)

#PID tuning parameters for V setpoint
Ku = 1.3
Tu = 1
CVCommands.P = 0.45*Ku    #3.06
CVCommands.I = 0.54*Ku/Tu    #2.8
CVCommands.D = 0.0

CVCommands.offset = 0.005 #Acceptable tolerance for voltage setpoint in V

CVCommands.reset_i = 0 #Subtract this value from voltage setpoint to improve PID speed
CVCommands.reset = 5 #Subtract value from setpoint after reset trigger

CVCommands.reset_val = 1000 #Trigger voltage setpoint reset after iterations
CVCommands.shutoff_val = 1000 #Max iterations before shutdown
CVCommands.num_avg = num_avg
CVCommands.mvavg_num = 20 #Number of points to calculate voltage setpoint moving average

CVCommands.v_ceiling = 35 #Max allowable voltage sent to SMU, maximum value 35

CVCommands.trig_delay = 0.1
CVCommands.meas_speed = 'SLOW2' #FAST, MED, SLOW, SLOW2
CVCommands.lowz = 'ON'
CVCommands.LCR_timeout = 60*1000


In [6]:
os.chdir('/home/pi/Desktop/DLCP')
foldname = datetime.today().strftime('%m-%d-%Y')
now = datetime.now()
currtime_format = now.strftime("%m/%d/%y %H:%M")

if os.path.isdir(foldname) is False:
    os.mkdir(foldname)

os.chdir(foldname)

try:
    DMM.query('*IDN?')
except:
    rm = pyvisa.ResourceManager()
    DMMdev = finddev(idVendor=0x05e6, idProduct=0x2100) #Device ID for DMM
    DMMdev.reset() 
    DMM = rm.open_resource('USB0::1510::8448::1182494::0::INSTR')
    LCR = rm.open_resource('GPIB0::17::INSTR')
    SMU = rm.open_resource('GPIB0::24::INSTR')
  
    CVCommands.LCR = LCR
    CVCommands.SMU = SMU
    CVCommands.DMM = DMM
    CVCommands.SMU_configure()
    CVCommands.playstair()
    CVCommands.playon()
    
CVCommands.sampName = sampName    
CVCommands.LCR_configure()
CVCommands.LCR_CGD()
error = False
time.sleep(1)
#Collect measurement (iAmpl = initial AC amplitude in V, fAmpl = final AC amplitude in V
#step_ampl = number of AC amplitude steps, num_avg = number of measurements to average over
#avg_delay = delay between measurements in s, delay = delay between changing conditions in s,
#measbias = the setpoint DC bias)
def takeMeasu(iAmpl,fAmpl,step_ampl):
    
    ampl_arr = []
    c_arr = []
    g_arr = []
    d_arr = []
    gw_arr = []
    
    for i in np.linspace(iAmpl,fAmpl,step_ampl):
        
        CVCommands.set_oscVolt(i)
        
        
        CVCommands.setBias(measbias-i,False,False)
        #print('Amplitude set to: '+str(LCR.query('VOLT?')))
        c, g, d = CVCommands.getData()

        ampl_arr.append(float(LCR.query(':MONI?').split(',')[0]))
        c_arr.append(c)
        g_arr.append(g)
        d_arr.append(d)

    tempdict = {
        'Amplitude (V)' : ampl_arr,
        'Capacitance (F)' : c_arr,
        'Dissipation (D)' : d_arr
    }

    output_df = pd.DataFrame(tempdict)
    
    return output_df


######################################################

if holdbias is True:

    CVCommands.setBias(hold_val,True,False)

    print('Prebias for ',hold_time,' seconds')
    time.sleep(hold_time)


fits = []
N_dl_4 = []
x_4 = []
V_arr=[]
C0=[]
C1=[]
C2=[]
amp_arr=[]
cap_arr=[]
freq_arr = []

##############################################

names_arr = []
try:
    for j in tqdm(np.logspace(freq_i,freq_f,freq_step),desc=''.join(['Freq sweep'])):

        CVCommands.setFreq(j)
        time.sleep(delay)
        print('Frequency set to: '+str(round(j/1000,2))+'kHz')
        fits = []
        #N_dl_1 = []
        #x_1 = []
        #N_dl_2 = []
        #x_2 = []
        #N_dl_3 = []
        #x_3 = []
        N_dl_4 = []
        x_4 = []
        V_arr = []
        C0 = []
        C1 = []
        C2 = []
        amp_arr = []
        cap_arr = []
        freq_arr = []
        d_arr=[]
        for i in tqdm(np.linspace(V_i,V_f,V_step),desc=''.join(['Bias sweep'])):

            CVCommands.setBias(i,True,False)

            measbias = float(DMM.query(':READ?'))

            time.sleep(delay)
            out = takeMeasu(ampl_i,ampl_f,ampl_step)

            fits = np.polyfit(out['Amplitude (V)'],out['Capacitance (F)'],2)
            #print(fit)
            #fits.append(fit)


    #         #def func1(x, c0, c1, c2):
    #             return c0 + c1*x + c2*x**2

    #         #def func2(x, c0, c1):
    #             return c0 + c1*x + 2*c1**2/c0*x**2

    #         #def func3(x, c0, c1):
    #             return c0 + c1*x + 2*c1**2/c0*x**2 + 5*c1**3/c0**2*x**3


            #popt1, pcov1 = curve_fit( func1, out['Amplitude (V)'], out['Capacitance (F)'], p0=fits, bounds=([-1e-5,0,-1e-5],[1e-5,1e-5,1e-5]))

            #popt2, pcov2 = curve_fit( func2, out['Amplitude (V)'], out['Capacitance (F)'], p0=fits[:-1], bounds=([-1e-5,0],[1e-5,1e-5]))

            #popt3, pcov3 = curve_fit( func3, out['Amplitude (V)'], out['Capacitance (F)'], p0=fits[:-1], bounds=([-1e-5,0],[1e-5,1e-5]))

            #print(popt1)
            #print(popt2)
            #print(popt3)
            #y_fit1 = func1(x, *popt1)
            #y_fit2 = func2(x, *popt2)
            #y_fit3 = func3(x, *popt3)        

            #N_dl_1.append(-popt1[0]**3/(2*q*ϵ*ϵo*A**2*popt1[1])*1e-6) #multiply by 10^-6 to get from m^-3 to cm^-3
            #x_1.append(ϵ*ϵo*A/popt1[0])

            #N_dl_2.append(-popt2[0]**3/(2*q*ϵ*ϵo*A**2*popt2[1])*1e-6) #multiply by 10^-6 to get from m^-3 to cm^-3
            #x_2.append(ϵ*ϵo*A/popt2[0])

            #N_dl_3.append(-popt3[0]**3/(2*q*ϵ*ϵo*A**2*popt3[1])*1e-6) #multiply by 10^-6 to get from m^-3 to cm^-3
            #x_3.append(ϵ*ϵo*A/popt3[0])

            N_dl_4.append(-fits[0]**3/(2*q*ϵ*ϵo*A**2*fits[1])*1e-6) #F3/V3/(C)/(m-3 kg-1 s4 C2 s-2)/(m^2)/(F)-->1/(C3.s2.m-1.kg-1) #multiply by 10^-6 to get from m^-3 to cm^-3
            x_4.append(ϵ*ϵo*A/fits[0]*1e9) #Multiply by 1e9 to get in nm

            C0.append(fits[0])
            C1.append(fits[1])
            C2.append(fits[2])

            amp_arr.append(out["Amplitude (V)"].tolist())
            cap_arr.append(out['Capacitance (F)'].tolist())
            d_arr.append(out['Dissipation (D)'].tolist())


            V_arr.append(measbias)
            freq_arr.append(LCR.query('FREQ?'))


        tempdict = {
            'Ndl' : N_dl_4,
            'x' : x_4,
            'Vreal' : V_arr,
            'freq' : freq_arr,
            'C0' : C0,
            'C1' : C1,
            'C2' : C2,
            'Ampl arr' : amp_arr,
            'Cap arr' : cap_arr,
            'D arr' : d_arr
        }

        info_labels = ['Sample Name: ','V start: ','V final: ','V steps: ','Freq start: ','Freq final: ','Freq steps: ','AC amp start: ','AC amp final: ','AC amp steps: ','Time: ']
        info_data = [sampName,V_i,V_f,V_step,10**freq_i,10**freq_f,freq_step,ampl_i,ampl_f,ampl_step,currtime_format]

        dictlen = len(tempdict[list(tempdict.keys())[0]])
        infolen = len(info_data)

        if dictlen >= infolen:
            info_labels = info_labels+[np.nan]*(dictlen-infolen)
            info_data = info_data+[np.nan]*(dictlen-infolen)
        else:
            for i in tempdict:
                tempdict[i] = tempdict[i]+[np.nan]*(infolen-dictlen)

        tempdict['Info1'] = info_labels
        tempdict['Info2'] = info_data

        print('Saving as: '+''.join([sampName,'_ndl',"{:.0f}".format(j/1000),'kHz']))
        globals()[''.join([sampName,'_ndl',"{:.0f}".format(j/1000),'kHz'])] = pd.DataFrame(tempdict)
        globals()[''.join([sampName,'_ndl',"{:.0f}".format(j/1000),'kHz'])].to_csv(''.join([sampName,'_ndl',"{:.0f}".format(j/1000),'kHz']),sep='\t')
        names_arr.append(''.join([sampName,'_ndl',"{:.0f}".format(j/1000),'kHz']))
except KeyboardInterrupt as e:
    CVCommands.offBias()
    error = True
    print(e)
    print('Interrupt')
CVCommands.offBias()
LCR.write(':TRIG INT')
print('Measurement complete')
print('Files saved: ')
print(names_arr)

if error == False:
    time.sleep(0.5)
    CVCommands.playchest()
else:
    CVCommands.playrand(10,0.05)

DC Bias set to 0.95V                                               

Prebias for  30  seconds


Bias sweep:   0%|          | 0/30 [00:00<?, ?it/s]

Frequency set to: 1.0kHz
DC Bias set to 0.95V                                            




Bias sweep:   3%|▎         | 1/30 [05:33<2:41:14, 333.60s/it]

DC Bias set to 0.917V                                           



...

DC Bias set to 0.033V                                             




Bias sweep:  97%|█████████▋| 29/30 [1:37:29<02:32, 152.42s/it]

DC Bias set to 0.0V                                                




Freq sweep: 100%|██████████| 5/5 [8:59:16<00:00, 6471.37s/it]  


Saving as: CS519C1_ndl100kHz
Measurement complete
Files saved: 
['CS519C1_ndl1kHz', 'CS519C1_ndl3kHz', 'CS519C1_ndl10kHz', 'CS519C1_ndl32kHz', 'CS519C1_ndl100kHz']
